In [1]:
!unzip asian_Dataset_jewelry.zip

Archive:  asian_Dataset_jewelry.zip
   creating: CV_Dataset_1800/
   creating: CV_Dataset_1800/TEST_300/
   creating: CV_Dataset_1800/TEST_300/Anklets/
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-001.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-002.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-003.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-004.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-005.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-006.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-007.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-008.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-009.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-010.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-011.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-012.jpg  
  inflating: CV_Dataset_1800/TEST_300/Anklets/Anklets-013.jpg  
  inflating: CV_

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

In [3]:
DATA_DIR = "/content/CV_Dataset_1800"

IMG_SIZE = (224, 224)   # EfficientNetB0
BATCH_SIZE = 32
SEED = 42

TRAIN_DIR = "/content/CV_Dataset_1800/TRAIN_1500"
TEST_DIR  = "/content/CV_Dataset_1800/TEST_300"

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(224, 224),
    batch_size=32,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)

Found 1500 files belonging to 6 classes.
Using 1200 files for training.
Found 1500 files belonging to 6 classes.
Using 300 files for validation.
Found 300 files belonging to 6 classes.
Classes: ['Anklets', 'Bangles', 'Earring', 'Necklace', 'Ring', 'Tikka']


In [14]:
from tensorflow.keras.applications.efficientnet import preprocess_input

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, training=False):
    ds = ds.map(
        lambda x, y: (preprocess_input(x), tf.one_hot(y, depth=num_classes)),
        num_parallel_calls=AUTOTUNE
    )
    if training:
        ds = ds.shuffle(1000)
    return ds.prefetch(AUTOTUNE)

In [15]:
train_ds = prepare(train_ds, training=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [16]:
for images, labels in train_ds.take(1):
    print(labels.shape)

(32, 6)


In [17]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SIZE + (3,)
)

base_model.trainable = False


In [18]:
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs, training=True)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)


In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 172ms/step - accuracy: 0.6905 - loss: 1.3039 - val_accuracy: 0.9433 - val_loss: 0.7056
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - accuracy: 0.9352 - loss: 0.7200 - val_accuracy: 0.9867 - val_loss: 0.6035
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.9567 - loss: 0.6671 - val_accuracy: 0.9900 - val_loss: 0.5605
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.9664 - loss: 0.6328 - val_accuracy: 0.9933 - val_loss: 0.5448
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 0.9795 - loss: 0.6057 - val_accuracy: 0.9933 - val_loss: 0.5258
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - accuracy: 0.9732 - loss: 0.5958 - val_accuracy: 0.9967 - val_loss: 0.4912
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 0.9877 - loss: 0.5573 - val_accuracy: 0.9933 - val_loss: 0.4894
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.9825 - loss: 0.5548 - val_accuracy: 0.9967 -

In [21]:
base_model.trainable = True

# Freeze bottom layers, fine-tune top 20
for layer in base_model.layers[:-30]:
    layer.trainable = False
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 24s 183ms/step - accuracy: 0.9273 - loss: 0.6695 - val_accuracy: 1.0000 - val_loss: 0.4673
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.9536 - loss: 0.6290 - val_accuracy: 1.0000 - val_loss: 0.4738
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.9560 - loss: 0.6146 - val_accuracy: 1.0000 - val_loss: 0.4806
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.9707 - loss: 0.5797 - val_accuracy: 1.0000 - val_loss: 0.4858
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.9799 - loss: 0.5704 - val_accuracy: 1.0000 - val_loss: 0.4898
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - accuracy: 0.9696 - loss: 0.5678 - val_accuracy: 1.0000 - val_loss: 0.4931
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.9715 - loss: 0.5639 - val_accuracy: 1.0000 - val_loss: 0.4949
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 112ms/step - accuracy: 0.9653 - loss: 0.5743 - val_accuracy: 1

In [24]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)


from sklearn.metrics import confusion_matrix, classification_report

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true = np.concatenate(
    [np.argmax(y, axis=1) for x, y in test_ds],
    axis=0
)

y_pred = np.argmax(model.predict(test_ds), axis=1)


print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=class_names))

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 335ms/step - accuracy: 0.8190 - loss: 0.8291
Test accuracy: 0.8966666460037231
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 355ms/step
[[33  0  0 17  0  0]
 [ 0 44  0  1  5  0]
 [ 0  0 50  0  0  0]
 [ 0  2  1 47  0  0]
 [ 0  0  1  0 49  0]
 [ 0  0  1  0  3 46]]
              precision    recall  f1-score   support

     Anklets       1.00      0.66      0.80        50
     Bangles       0.96      0.88      0.92        50
     Earring       0.94      1.00      0.97        50
    Necklace       0.72      0.94      0.82        50
        Ring       0.86      0.98      0.92        50
       Tikka       1.00      0.92      0.96        50

    accuracy                           0.90       300
   macro avg       0.91      0.90      0.90       300
weighted avg       0.91      0.90      0.90       300

